In [6]:
from car_part import CarPartDataset, CarPartConfig, prepare_datasets, preprocess_dataset
from pathlib import Path
from mrcnn import visualize
import colorsys
import random
import numpy as np
import mrcnn.model as modellib
from mrcnn.model import log
import scipy.io as sio
import os
import json

import matplotlib.pyplot as plt
plt.ion()

# How to train the model

In [2]:
part_annotation_files = Path(
    '../../../car-damage-estimation/data/car_part/trainval/Annotations_Part/')
images_path = Path('../../../car-damage-estimation/data/VOCdevkit/VOC2010/JPEGImages/')

images_annotations_files = list(Path(part_annotation_files).glob('*.mat'))

## Prepare the datasets

In [3]:
dataset_train, dataset_val, dataset_test, parts_idx_dict = prepare_datasets(images_path, images_annotations_files, 0.9, 1.0)

100%|██████████| 10103/10103 [01:57<00:00, 85.99it/s] 

train size 919, test size 1022


list(zip(dataset_train.class_names, dataset_train.class_ids))

In [ ]:
print(dataset_train.num_images, dataset_val.num_images, dataset_test.num_images)

In [4]:
parts_idx_dict

{'backside': 1,
 'bliplate': 2,
 'door_1': 3,
 'door_2': 4,
 'door_3': 5,
 'fliplate': 6,
 'frontside': 7,
 'headlight_1': 8,
 'headlight_2': 9,
 'headlight_3': 10,
 'headlight_4': 11,
 'headlight_5': 12,
 'headlight_6': 13,
 'leftmirror': 14,
 'leftside': 15,
 'rightmirror': 16,
 'rightside': 17,
 'roofside': 18,
 'wheel_1': 19,
 'wheel_2': 20,
 'wheel_3': 21,
 'wheel_4': 22,
 'wheel_5': 23,
 'window_1': 24,
 'window_2': 25,
 'window_3': 26,
 'window_4': 27,
 'window_5': 28,
 'window_6': 29,
 'window_7': 30}

In [8]:
with open('parts_idx_dict.json', 'w') as f:
    json.dump(parts_idx_dict, f)

In [10]:
!cat parts_idx_dict.json

{"backside": 1, "bliplate": 2, "door_1": 3, "door_2": 4, "door_3": 5, "fliplate": 6, "frontside": 7, "headlight_1": 8, "headlight_2": 9, "headlight_3": 10, "headlight_4": 11, "headlight_5": 12, "headlight_6": 13, "leftmirror": 14, "leftside": 15, "rightmirror": 16, "rightside": 17, "roofside": 18, "wheel_1": 19, "wheel_2": 20, "wheel_3": 21, "wheel_4": 22, "wheel_5": 23, "window_1": 24, "window_2": 25, "window_3": 26, "window_4": 27, "window_5": 28, "window_6": 29, "window_7": 30}

### Check some images

In [ ]:
colors = visualize.generate_colormap(30, scaled=False)

In [ ]:
i = 2

In [ ]:
img = dataset_train.load_image(i)
plt.imshow(img)

In [ ]:
mask, classes = dataset_train.load_mask(i)
img_copy = img.copy()

for l in range(mask.shape[-1]):
    img_copy = visualize.apply_mask(img_copy, mask[:,:,l], colors[l])
    
plt.imshow(img_copy)

In [ ]:
[dataset_val.class_names[c] for c in classes]

## Create the model

In [ ]:
def make_subplot(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
config = CarPartConfig()
MODEL_DIR = './logs/'
config.BATCH_SIZE

## Inspect the model results

In [ ]:
model = modellib.MaskRCNN(mode="inference", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
model.find_last()

In [ ]:
model.load_weights(model.find_last(), by_name=True)

In [ ]:
!ls ./logs/car_parts20190326T1032

In [ ]:
model.load_weights('./logs/car_parts20190326T1032/mask_rcnn_car_parts_0040.h5', by_name=True)

In [ ]:
dataset_train.class_names

In [ ]:
result['class_ids']

In [ ]:
#select an image
img = dataset_train.load_image(2)
result = model.detect([img])[0]
ax = make_subplot(rows=1, size=12)

img = visualize.display_instances(img, result['rois'], result['masks'], result['class_ids'], 
                            dataset_train.class_names, colors, result['scores'], ax=ax,
                            title="Predictions")